In [44]:
# Imports
import pandas as pd
from typing import Annotated
from pydantic import BaseModel, Field, HttpUrl, PositiveInt, ValidationError
import great_expectations as gx

# gx context for the data quality tests
context = gx.get_context(mode='file')


# Options
pd.set_option('display.max_columns', 100)

# Utils Functions

In [45]:
# Data Validation Model
class MoviesMetadataModel(BaseModel):

    color : str
    director_name: str
    num_critic_for_reviews: Annotated[int, Field(strict=False, ge=0)]
    duration: Annotated[float, Field(strict=False, ge=0)]
    director_facebook_likes: Annotated[int, Field(strict=False, ge=0)]
    actor_3_facebook_likes: Annotated[int, Field(strict=False, ge=0)]
    actor_2_name: str
    actor_1_facebook_likes: Annotated[int, Field(strict=False, ge=0)]
    gross: Annotated[float, Field(strict=False, ge=0)]
    genres: str
    actor_1_name: str
    movie_title: str
    num_voted_users: Annotated[int, Field(strict=False, ge=0)]
    cast_total_facebook_likes: Annotated[int, Field(strict=False, ge=0)]
    actor_3_name: str
    facenumber_in_poster: Annotated[int, Field(strict=False, ge=0)]
    plot_keywords: str
    movie_imdb_link: HttpUrl
    num_user_for_reviews: PositiveInt
    language: str
    country:str
    content_rating: str
    budget: Annotated[float, Field(strict=False, ge=0)]
    title_year: PositiveInt
    actor_2_facebook_likes: int
    imdb_score: Annotated[float, Field(strict=False, ge=0)]
    aspect_ratio: Annotated[float, Field(strict=False, ge=0)]
    movie_facebook_likes: Annotated[int, Field(strict=False, ge=0)]


In [46]:
def percentage_missing(data):
    """
    """
    
    missing = data.isna().sum()
    total = data.shape[0]
    percentage = 100*(missing/total)
    
    return percentage

def del_spaces(data:pd.DataFrame)->pd.DataFrame:

    """
    """
    if isinstance(data, str):
        return data.strip()
    return data

def del_duplicates(data:pd.DataFrame)->pd.DataFrame:

    """
    """
    data=data.drop_duplicates()

    return data

def fill_missing_zero(data:pd.DataFrame, columns_to_fill:list)->pd.DataFrame:

    """
    """
    data[columns_to_fill]=data[columns_to_fill].fillna(0)

    # Filling Color
    data.loc[(data['color'].isna())&(data['title_year']>=1970), 'color'] = 'Color'

    return data

def drop_nulls(data:pd.DataFrame)->pd.DataFrame:

    """
    """
    data=data.dropna()
    return data

# 1. Data Loading

What I like to do is start with the simplest solution to the problem (in this case, loading the data), and then, as needed, move on to more complex approaches. With that in mind, and since a .csv file was provided, I would begin by trying a straightforward pd.read_csv(). The file wasn’t too large, so I could visually inspect it beforehand and confirm that a comma was used as the delimiter and the first row was the header—hence the use of sep=',' and header=0.

In this instance, the pd.read_csv method worked, but what could have gone wrong? If the file had been too large, I would have considered other options like Dask, Polars, or even PySpark.

After the initial inspection performed in section 1.1, we identified that our dataset contains both numerical and categorical variables, and that applying some type of schema enforcement would be beneficial. We also discovered the presence of outliers and missing values. We will handle the missing values, but the outliers will be addressed by the data science team based on the specific business problem they are tackling.

In [47]:
# Loading data.
data = pd.read_csv('data/external/movie_metadata.csv', sep=',', header=0, skipinitialspace=True)
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


### 1.1 Initial Inspection

For the initial inspection, besides looking at the head of the DataFrame, I would perform the actions below to get a better sense of the data:

**1. Inspect the shape.**

Just to understand the number of rows and columns (dimensionality) we’re working with.

**2. Inspect the data types.**

By checking the data types, we can determine whether some schema enforcement is needed. This is usually a good practice, so we’ll include schema validation in the pipeline.

**3. Check statistical values.**

Here we detected outliers by comparing the 75th percentile with the max value of some columns. Since the difference was large, we could infer the presence of outliers—for example, in the director_facebook_likes column.

**4. Check for unique values.**

The goal here is to get a sense of the spread or capillarity of the variables. Everything seemed normal in this regard.

**5. Check for missing values.**

By analyzing the percentage of missing data, we can understand how complete our dataset is. The variables gross and budget had the highest percentages of missing values. Since these are financial fields, we could consider filling them with zero and maybe drop the remaining missing entries after discussions with the data science and business teams.

**6. Check for duplicates.**

We found just 45 duplicate rows we can just drop them.

**7. Low variance and constant columns.**

We didn’t find any columns with low variance or constant values. This check is important because such columns contribute little to no information to the models while still incurring storage and processing costs.

In [48]:
# Inspect the shape of dataframe.
# 5043 rows and 28 columns
data.shape

(5043, 28)

In [49]:
# Inspect the data types of the columns and the size of dataframe.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

In [50]:
# check statistical values for the data
data.describe(include='all')

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,5024,4939,4993.000000,5028.000000,4939.000000,5020.000000,5030,5036.000000,4.159000e+03,5043,5036,5043,5.043000e+03,5043.000000,5020,5030.000000,4890,5043,5022.000000,5029,5038,4740,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
unique,2,2398,NaN,NaN,NaN,NaN,3032,NaN,NaN,914,2097,4917,NaN,NaN,3521,NaN,4760,4919,NaN,46,65,18,NaN,NaN,NaN,NaN,NaN,NaN
top,Color,Steven Spielberg,NaN,NaN,NaN,NaN,Morgan Freeman,NaN,NaN,Drama,Robert De Niro,King Kong,NaN,NaN,Steve Coogan,NaN,based on novel,http://www.imdb.com/title/tt0232500/?ref_=fn_t...,NaN,English,USA,R,NaN,NaN,NaN,NaN,NaN,NaN
freq,4815,26,NaN,NaN,NaN,NaN,20,NaN,NaN,236,49,3,NaN,NaN,8,NaN,4,3,NaN,4704,3807,2118,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,140.194272,107.201074,686.509212,645.009761,NaN,6560.047061,4.846841e+07,NaN,NaN,NaN,8.366816e+04,9699.063851,NaN,1.371173,NaN,NaN,272.770808,NaN,NaN,NaN,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,NaN,NaN,121.601675,25.197441,2813.328607,1665.041728,NaN,15020.759120,6.845299e+07,NaN,NaN,NaN,1.384853e+05,18163.799124,NaN,2.013576,NaN,NaN,377.982886,NaN,NaN,NaN,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,NaN,NaN,1.000000,7.000000,0.000000,0.000000,NaN,0.000000,1.620000e+02,NaN,NaN,NaN,5.000000e+00,0.000000,NaN,0.000000,NaN,NaN,1.000000,NaN,NaN,NaN,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,NaN,NaN,50.000000,93.000000,7.000000,133.000000,NaN,614.000000,5.340988e+06,NaN,NaN,NaN,8.593500e+03,1411.000000,NaN,0.000000,NaN,NaN,65.000000,NaN,NaN,NaN,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,NaN,NaN,110.000000,103.000000,49.000000,371.500000,NaN,988.000000,2.551750e+07,NaN,NaN,NaN,3.435900e+04,3090.000000,NaN,1.000000,NaN,NaN,156.000000,NaN,NaN,NaN,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,NaN,NaN,195.000000,118.000000,194.500000,636.000000,NaN,11000.000000,6.230944e+07,NaN,NaN,NaN,9.630900e+04,13756.500000,NaN,2.000000,NaN,NaN,326.000000,NaN,NaN,NaN,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000


In [51]:
# check for unique values
unique_report = {col:data[col].nunique() for col in data.columns}
unique_report

{'color': 2,
 'director_name': 2398,
 'num_critic_for_reviews': 528,
 'duration': 191,
 'director_facebook_likes': 435,
 'actor_3_facebook_likes': 906,
 'actor_2_name': 3032,
 'actor_1_facebook_likes': 878,
 'gross': 4035,
 'genres': 914,
 'actor_1_name': 2097,
 'movie_title': 4917,
 'num_voted_users': 4826,
 'cast_total_facebook_likes': 3978,
 'actor_3_name': 3521,
 'facenumber_in_poster': 19,
 'plot_keywords': 4760,
 'movie_imdb_link': 4919,
 'num_user_for_reviews': 954,
 'language': 46,
 'country': 65,
 'content_rating': 18,
 'budget': 439,
 'title_year': 91,
 'actor_2_facebook_likes': 917,
 'imdb_score': 78,
 'aspect_ratio': 22,
 'movie_facebook_likes': 876}

In [52]:
# Check for missing values
percentage_missing(data=data)

color                         0.376760
director_name                 2.062265
num_critic_for_reviews        0.991473
duration                      0.297442
director_facebook_likes       2.062265
actor_3_facebook_likes        0.456078
actor_2_name                  0.257783
actor_1_facebook_likes        0.138806
gross                        17.529248
genres                        0.000000
actor_1_name                  0.138806
movie_title                   0.000000
num_voted_users               0.000000
cast_total_facebook_likes     0.000000
actor_3_name                  0.456078
facenumber_in_poster          0.257783
plot_keywords                 3.033908
movie_imdb_link               0.000000
num_user_for_reviews          0.416419
language                      0.277613
country                       0.099147
content_rating                6.008328
budget                        9.756098
title_year                    2.141582
actor_2_facebook_likes        0.257783
imdb_score               

In [53]:
# Duplicates Checks
duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 45


In [54]:
# Low variance and constant columns
low_variance_cols = [col for col in data.columns if data[col].nunique() <= 1]
print("Low variance columns:", low_variance_cols)

Low variance columns: []


### 1.2 Data Validation

Here we’re running a data validation test, and as shown in the report below, it’s failing exactly due to the missing values.

For now, that’s fine. We’ll handle those later in the pipeline. However, before any write operation, the dataset must pass this validation check.

In [55]:
import csv
import warnings

with open('data/external/movie_metadata.csv') as f:
    reader = csv.DictReader(f)
    try:
        data_parsed = [MoviesMetadataModel.model_validate(row) for row in reader]
    except ValidationError as exec:
        warnings.warn(f""" Validation Report: {exec}""")

C:\Users\ACER\AppData\Local\Temp\ipykernel_6892\1644801588.py:9: UserWarning:  Validation Report: 8 validation errors for MoviesMetadataModel
num_critic_for_reviews
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
duration
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing
actor_3_facebook_likes
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
gross
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/

# 2. Initial Data Quality Assessment

**The Plan:**

1. Schema:

    We are going to test the set of columns names to guarante the dimensions integrity of the dataset.
    The problem we are trying to avoid here is in case of a third party provider include some new column/dimension without previous notice 
    our pipeline won't broke because of it.
    About the types we are going to test some possibles types (e.g string and integer) per column, recalling that in task one we build a schema model that we can use to enforce it when we want. 

2. Uniqueness:

    Here we are going to test for a column if a unique indentifier, since we dont have from our dataset we will user a compound value of the columns 
    movie_imdb_link, movie_title. I'm assuming that every movie_title should have just one link. 
    
    Why it's important?
    Without a unique indentifier we wont be able to join with others tables and data sources what impact the task 5 or any try of enhance the dataset.

3. Volume:

    At volume i want to test how much data we are dealing with by testing the total count of rows.
    
    Why it's important?
    Let's say we are consuming data from an API and there is some spike in the volume of data provided by the API, it can happen because some erro a lot of duplicate values for example or because the provider increased the size of the batch.
    If we are not ready to take that amount of data it could cause delays in the pipeline execution or even broke it (e.g memory problems)  

4. Missing:

    Here we want to indentify the columns if higher percentage of missing i try to come up with strategys for each one.
    Will be considered as failed missing pecentage over 5% because lower then that we could just drop since the data loss will be minimum. 

**Results:**

From our tests just the missing values failed specifically the columns groos, budget, content_rating and aspect_ratio.
The strategy to deal with them is going to be fill with 0 then after try to found this information on a external source and enrich the dataset. The columns with less then 5% missing values we will drop them.
During the tests i also noticed that some values of the dataframe has spaces in it which i not ideal so we wiil have functions to deal with it and eleminated duplidated entries too.

## 2.1 Config Great Expectations data source, asset, batch and expectations.

In [56]:
# Defining data source, asset and Batch
try:
    data_source = context.data_sources.add_pandas("pandas")
except:
    data_source = context.data_sources.get("pandas")

try:
    data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")
except:
    data_asset = data_source.get_asset(name="pd dataframe asset")

try:
    batch_definition = data_asset.add_batch_definition_whole_dataframe("batch_definition")
except:
    batch_definition = data_asset.get_batch_definition("batch_definition")

batch = batch_definition.get_batch(batch_parameters={"dataframe":data})

In [79]:
# Create the expectations

# Schema
expectation_column_name = gx.expectations.ExpectTableColumnsToMatchSet(column_set=['color',
                                                                                   'director_name',
                                                                                   'num_critic_for_reviews',
                                                                                   'duration',
                                                                                   'director_facebook_likes',
                                                                                   'actor_3_facebook_likes',
                                                                                   'actor_2_name',
                                                                                   'actor_1_facebook_likes',
                                                                                   'gross',
                                                                                   'genres',
                                                                                   'actor_1_name',
                                                                                   'movie_title',
                                                                                   'num_voted_users',
                                                                                   'cast_total_facebook_likes',
                                                                                   'actor_3_name',
                                                                                   'facenumber_in_poster',
                                                                                   'plot_keywords',
                                                                                   'movie_imdb_link',
                                                                                   'num_user_for_reviews',
                                                                                   'language',
                                                                                   'country',
                                                                                   'content_rating',
                                                                                   'budget',
                                                                                   'title_year',
                                                                                   'actor_2_facebook_likes',
                                                                                   'imdb_score',
                                                                                   'aspect_ratio',
                                                                                   'movie_facebook_likes'])


# Uniqueness
expectation_unique_1 = gx.expectations.ExpectCompoundColumnsToBeUnique(column_list=["movie_title","title_year", "num_voted_users","cast_total_facebook_likes"])

#movie_imdb_link
expectation_unique_2 = gx.expectations.ExpectColumnValuesToBeUnique(column='movie_imdb_link')


# Volume
expectation_row_count = gx.expectations.ExpectTableRowCountToBeBetween(min_value=4500 ,  max_value=6000)

# Missing values - expectations is in section 2.2.4

## 2.2 Evaluating expectations

### 2.2.1 Evaluating schemas expectations

In [58]:
# Evaluating Names of the columns
results = batch.validate(expectation_column_name)
print(results)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics: 100%|██████████| 2/2 [00:00<00:00, 24.62it/s] 

{
  "success": true,
  "expectation_config": {
    "type": "expect_table_columns_to_match_set",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column_set": [
        "color",
        "director_name",
        "num_critic_for_reviews",
        "duration",
        "director_facebook_likes",
        "actor_3_facebook_likes",
        "actor_2_name",
        "actor_1_facebook_likes",
        "gross",
        "genres",
        "actor_1_name",
        "movie_title",
        "num_voted_users",
        "cast_total_facebook_likes",
        "actor_3_name",
        "facenumber_in_poster",
        "plot_keywords",
        "movie_imdb_link",
        "num_user_for_reviews",
        "language",
        "country",
        "content_rating",
        "budget",
        "title_year",
        "actor_2_facebook_likes",
        "imdb_score",
        "aspect_ratio",
        "movie_facebook_likes"
      ]
    },
    "meta": {}
  },
  "result": {
    "observed_value": [
      "color",
      

In [59]:
#ExpectColumnValuesToBeInTypeList

### 2.2.2 Evaluating uniqueness expectations.

In [60]:
# Evaluating Uniqueness of the columns
results = batch.validate(expectation_unique_1)
print(results)

Calculating Metrics: 100%|██████████| 9/9 [00:00<00:00, 289.39it/s] 

{
  "success": false,
  "expectation_config": {
    "type": "expect_compound_columns_to_be_unique",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column_list": [
        "movie_title",
        "title_year",
        "num_voted_users",
        "cast_total_facebook_likes"
      ]
    },
    "meta": {}
  },
  "result": {
    "element_count": 5043,
    "unexpected_count": 90,
    "unexpected_percent": 1.784651992861392,
    "partial_unexpected_list": [
      {
        "movie_title": "The Avengers\u00a0",
        "title_year": 2012.0,
        "num_voted_users": 995415,
        "cast_total_facebook_likes": 87697
      },
      {
        "movie_title": "The Legend of Tarzan\u00a0",
        "title_year": 2016.0,
        "num_voted_users": 42372,
        "cast_total_facebook_likes": 21175
      },
      {
        "movie_title": "Godzilla Resurgence\u00a0",
        "title_year": 2016.0,
        "num_voted_users": 374,
        "cast_total_facebook_likes": 699
      },
     

In [109]:
# Evaluating Uniqueness of the columns
results_unique_2 = batch.validate(expectation_unique_2)
print(results_unique_2)

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 99.49it/s] 

{
  "success": false,
  "expectation_config": {
    "type": "expect_column_values_to_be_unique",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column": "movie_imdb_link"
    },
    "meta": {}
  },
  "result": {
    "element_count": 5043,
    "unexpected_count": 241,
    "unexpected_percent": 4.7789014475510605,
    "partial_unexpected_list": [
      "http://www.imdb.com/title/tt0413300/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt0848228/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt0360717/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1074638/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1014759/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1623205/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1104001/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1343092/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt0918940/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt3040964/?ref_=fn_tt_tt_1",
      "htt

### 2.2.3 Evaluating volume expectations.

In [61]:
# Evaluating the row count of the batch data
results = batch.validate(expectation_row_count)
print(results)

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 134.23it/s]

{
  "success": true,
  "expectation_config": {
    "type": "expect_table_row_count_to_be_between",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "min_value": 4500,
      "max_value": 6000
    },
    "meta": {}
  },
  "result": {
    "observed_value": 5043
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


### 2.2.4 Evaluating Missing Values

In [62]:
# Evaluating Missingness
validation_results_by_columns = []

for column in data.columns:
    expectation_null = gx.expectations.ExpectColumnValuesToNotBeNull(column=column, mostly = 0.95)
    results = batch.validate(expectation_null)
    validation_results_by_columns.append(
        {
            "Validation":results['success'],
            "Columns":results['expectation_config']['kwargs']['column'],
            "Element_count":results['result']['element_count'],
            "Unexpected_count":results['result']['unexpected_count'],
            "Unexpected_percent":results['result']['unexpected_percent']
        }
    )

df = pd.DataFrame(validation_results_by_columns)
df.loc[df['Validation']==False, : ]

Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 511.89it/s]


,Validation,Columns,Element_count,Unexpected_count,Unexpected_percent
8,False,gross,5043,884,17.529248
21,False,content_rating,5043,303,6.008328
22,False,budget,5043,492,9.756098
26,False,aspect_ratio,5043,329,6.523895


### 2.2.5 Transformations

In [63]:
data = drop_nulls(
            fill_missing_zero(
                 del_duplicates(
                     del_spaces(data=data)), columns_to_fill=['gross','budget','content_rating', 'aspect_ratio']))

C:\Users\ACER\AppData\Local\Temp\ipykernel_6892\720879212.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_fill]=data[columns_to_fill].fillna(0)


In [64]:
data.isna().sum()

color                        0
director_name                0
num_critic_for_reviews       0
duration                     0
director_facebook_likes      0
actor_3_facebook_likes       0
actor_2_name                 0
actor_1_facebook_likes       0
gross                        0
genres                       0
actor_1_name                 0
movie_title                  0
num_voted_users              0
cast_total_facebook_likes    0
actor_3_name                 0
facenumber_in_poster         0
plot_keywords                0
movie_imdb_link              0
num_user_for_reviews         0
language                     0
country                      0
content_rating               0
budget                       0
title_year                   0
actor_2_facebook_likes       0
imdb_score                   0
aspect_ratio                 0
movie_facebook_likes         0
dtype: int64

# 3. Numeric Data Consistency

**The Plan:**

   We are going to aplly the following tests:

   **Consitency:**
   
   z_score Test : 
      
   We are going to use ExpectColumnValueZScoresToBeLessThan from GE library a powerful expectation for identifying individual outliers and anomalous data points that could represent data entry issues or unusual transactions.

   Values inside a range test:  
      
   Essential for bounding numerical values within valid ranges, such as likes can not bi negatives
   

**Results:**

   In the z_score we found out the 15 colummns has values above the third standard deviantion, it doesn't mean that somenthing is wrong but futher invetigation with the business team is required.

   About range values test, the objetive was to test the *likes columns for negative numbers what wouldn't make sense since there is no negative like. All columns were just positives and so everything is fine.


In [65]:
# Evaluating z-score
validation_results_by_columns = []

for column in data.describe().columns:
    expectation_z_score = gx.expectations.ExpectColumnValueZScoresToBeLessThan(column=column, threshold=3 , double_sided=True)
    results = batch.validate(expectation_z_score)
    validation_results_by_columns.append(
        {
            "Validation":results['success'],
            "Columns":results['expectation_config']['kwargs']['column'],
            "Element_count":results['result']['element_count'],
            "Unexpected_count":results['result']['unexpected_count'],
            "Unexpected_percent":results['result']['unexpected_percent'],
            "Partial_unexpected_list":results['result']['partial_unexpected_list'],
            "Partial_unexpected_index_list":results['result']['partial_unexpected_index_list']
        }
    )

df = pd.DataFrame(validation_results_by_columns)
report_zscore = df.loc[df['Validation']==False, : ]


Calculating Metrics: 100%|██████████| 13/13 [00:00<00:00, 415.90it/s]


In [66]:
# Evaluating ranges of likes columns
import re
like_columns = [item for item in data.describe().columns if re.search("like", item)]

validation_results_by_columns = []

for column in like_columns:
    expectation_value_between = gx.expectations.ExpectColumnValuesToBeBetween(column=column, min_value=0 , max_value=500000)
    results = batch.validate(expectation_value_between)
    validation_results_by_columns.append(
        {
            "Validation":results['success'],
            "Columns":results['expectation_config']['kwargs']['column'],
            "Element_count":results['result']['element_count'],
            "Unexpected_count":results['result']['unexpected_count'],
            "Unexpected_percent":results['result']['unexpected_percent'],
            "Partial_unexpected_list":results['result']['partial_unexpected_list'],
            "Partial_unexpected_index_list":results['result']['partial_unexpected_index_list']
            
        }
    )

df = pd.DataFrame(validation_results_by_columns)
report = df.loc[df['Validation']==False, : ]

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 320.00it/s]


In [67]:
report

,Validation,Columns,Element_count,Unexpected_count,Unexpected_percent,Partial_unexpected_list,Partial_unexpected_index_list
2,False,actor_1_facebook_likes,5043,1,0.019857,[640000.0],[1902]
3,False,cast_total_facebook_likes,5043,1,0.019829,[656730],[1902]


In [68]:
# Other columns that can not be negative
non_negative_columns = ['title_year', 'imdb_score', 'aspect_ratio']

validation_results_by_columns = []

for column in non_negative_columns:
    expectation_value_between = gx.expectations.ExpectColumnValuesToBeBetween(column=column, min_value=0 , max_value=500000)
    results = batch.validate(expectation_value_between)
    validation_results_by_columns.append(
        {
            "Validation":results['success'],
            "Columns":results['expectation_config']['kwargs']['column'],
            "Element_count":results['result']['element_count'],
            "Unexpected_count":results['result']['unexpected_count'],
            "Unexpected_percent":results['result']['unexpected_percent'],
            "Partial_unexpected_list":results['result']['partial_unexpected_list'],
            "Partial_unexpected_index_list":results['result']['partial_unexpected_index_list']
            
        }
    )

df = pd.DataFrame(validation_results_by_columns)
report = df.loc[df['Validation']==False, : ]

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 305.28it/s]


In [69]:
report

,Validation,Columns,Element_count,Unexpected_count,Unexpected_percent,Partial_unexpected_list,Partial_unexpected_index_list


# 4. Data Integrity and Duplicates

Plan:

    Duplicated:

    We already checked for duplicates in the Loading session.
    Here we put in evidence items found in section that are no that obvious that are duplicated.

    Integrity:

    Check the relations between the color column and title_year, in order to verify if any modern movie is labeled as Black and White or the opposite.



### 4.1 Duplicated

During the test of uniqueness in section 2.2.2 we found some duplicated items. During futher investigation we found items that would be identical if not by a specific field.

Example indexes 6 and 3461 they are diferente just for the field num_voted_users. This kind of problem is hard to threat without business knowledge! What would be the right value 383056 or 383071.


In [89]:
#Item found in the section 2.2.2 during  
data.loc[data['plot_keywords']=='sandman|spider man|symbiote|venom|villain', :]

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,J.K. Simmons,Spider-Man 3,383056,46055,Kirsten Dunst,0.0,sandman|spider man|symbiote|venom|villain,http://www.imdb.com/title/tt0413300/?ref_=fn_t...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
3461,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,J.K. Simmons,Spider-Man 3,383071,46055,Kirsten Dunst,0.0,sandman|spider man|symbiote|venom|villain,http://www.imdb.com/title/tt0413300/?ref_=fn_t...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0


In [ ]:
# Report from section 2.2.2 - testing the uniqueness of movie_imdb_link
results_unique_2

{
  "success": false,
  "expectation_config": {
    "type": "expect_column_values_to_be_unique",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column": "movie_imdb_link"
    },
    "meta": {}
  },
  "result": {
    "element_count": 5043,
    "unexpected_count": 241,
    "unexpected_percent": 4.7789014475510605,
    "partial_unexpected_list": [
      "http://www.imdb.com/title/tt0413300/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt0848228/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt0360717/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1074638/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1014759/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1623205/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1104001/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt1343092/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt0918940/?ref_=fn_tt_tt_1",
      "http://www.imdb.com/title/tt3040964/?ref_=fn_tt_tt_1",
      "htt

### 4.2 Integrity

After 1960s color films become widely adopted so i decided to adopt 1970 as a threshold year. Was found 137 movies erroneous labeled as Black and White and was found 93 movies erroneous labeled as Color.



In [97]:
#All movies labeled as Black and White. After 1960s color films become widely adopted.
wrong_bw = data.loc[(data['color']=='Black and White') & (data['title_year']>1970), ['color','movie_title','title_year']]
#len(wrong_bw['movie_title'])
wrong_bw.head()


,color,movie_title,title_year
111,Black and White,Pearl Harbor,2001.0
149,Black and White,Die Another Day,2002.0
257,Black and White,The Aviator,2004.0
272,Black and White,Ali,2001.0
286,Black and White,Casino Royale,2006.0


In [99]:
wrong_cl = data.loc[(data['color']=='Color') & (data['title_year']<=1970), ['color','movie_title','title_year']]
#len(wrong_cl['movie_title'])
wrong_cl.head()

,color,movie_title,title_year
1099,Color,"Yours, Mine and Ours",1968.0
1143,Color,Pinocchio,1940.0
1160,Color,Cleopatra,1963.0
1933,Color,Tora! Tora! Tora!,1970.0
2013,Color,Waterloo,1970.0


In [116]:
# Transformation to change the color labeling 

def color_or_black_and_white(data):

     data.loc[(data['color']=='Black and White') & (data['title_year']>1970), ['color']] = 'Color'
     data.loc[(data['color']=='Color') & (data['title_year']<=1970), ['color']] = 'Black and White'

     return data

data = color_or_black_and_white(data=data)

# 5. Data Enrichment

# 6. Data Transformation and Standardization

# 7. Pipeline Design

# 8. Bonus Challenge Question - Actor Collaboration and Performance Analysis